In [11]:
import time
import numpy as np
from typing import Tuple, List, Dict

from scipy import sparse as sp
from scipy.linalg import eigh          # dense solver
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh  # sparse solver


# -----------------------------
# Graph generator (reproducible)
# -----------------------------
def generate_layers_groups_graph(
    num_supergroups: int = 2,
    num_subgroups_per_supergroup: int = 2,
    nodes_per_subgroup: int = 5,
    p_intra_subgroup: float = 0.8,
    p_intra_supergroup: float = 0.3,
    p_inter_supergroup: float = 0.05,
    seed: int | None = 841,    # global seed for reproducibility (None => random)
) -> np.ndarray:
    """Generate hierarchical Laplacian L (dense, float64)."""
    if seed is not None:
        np.random.seed(seed)

    nsg, nps = num_subgroups_per_supergroup, nodes_per_subgroup
    N = num_supergroups * nsg * nps
    A = np.zeros((N, N), dtype=np.uint8)

    def idx(g, sg, k):
        return (g * nsg + sg) * nps + k

    # Intra-supergroup
    for g in range(num_supergroups):
        for sg1 in range(nsg):
            # same subgroup
            for k1 in range(nps):
                i = idx(g, sg1, k1)
                for k2 in range(k1 + 1, nps):
                    j = idx(g, sg1, k2)
                    if np.random.rand() < p_intra_subgroup:
                        A[i, j] = A[j, i] = 1
            # different subgroups in same supergroup
            for sg2 in range(sg1 + 1, nsg):
                for k1 in range(nps):
                    i = idx(g, sg1, k1)
                    for k2 in range(nps):
                        j = idx(g, sg2, k2)
                        if np.random.rand() < p_intra_supergroup:
                            A[i, j] = A[j, i] = 1

    # Inter-supergroup
    for g1 in range(num_supergroups):
        for g2 in range(g1 + 1, num_supergroups):
            for sg1 in range(nsg):
                for sg2 in range(nsg):
                    for k1 in range(nps):
                        i = idx(g1, sg1, k1)
                        for k2 in range(nps):
                            j = idx(g2, sg2, k2)
                            if np.random.rand() < p_inter_supergroup:
                                A[i, j] = A[j, i] = 1

    deg = A.sum(axis=1, dtype=np.int64).astype(np.float64)
    L = np.diag(deg) - A.astype(np.float64)
    return L


# -----------------------------------------
# Fiedler vector by dense or sparse backend
# -----------------------------------------
def _fiedler_vector(L, solver: str = "sparse") -> np.ndarray:
    """
    Return Fiedler vector of Laplacian L using:
      - solver='dense'  -> scipy.linalg.eigh (subset_by_index)
      - solver='sparse' -> scipy.sparse.linalg.eigsh
    L can be dense ndarray or sparse CSR; we normalize as needed.
    """
    if solver not in ("dense", "sparse"):
        raise ValueError("solver must be 'dense' or 'sparse'.")

    n = L.shape[0]

    if solver == "dense":
        # Convert to dense (if sparse), then compute the two smallest eigenpairs
        A = L.toarray() if sp.issparse(L) else L
        # eigh with subset_by_index avoids full spectrum
        # Note: check_finite=False for speed (assumes inputs are finite)
        w, v = eigh(A, subset_by_index=[0, 1], check_finite=False, overwrite_a=False)
        return v[:, 1]

    # sparse:
    Lsp = L if sp.issparse(L) else csr_matrix(L)
    # float32 often suffices and is faster; switch to float64 if convergence issues
    Lsp = Lsp.astype(np.float32)
    vals, vecs = eigsh(Lsp, k=2, which='SA')
    return vecs[:, 1]


# ---------------------------
# bicut (with solver choice)
# ---------------------------
def bicut_group(L, solver: str = "sparse") -> Tuple[np.ndarray, np.ndarray]:
    """
    Your original bicut, but with a 'solver' switch for dense/sparse eigen solve.
    Returns local index lists (first_group, second_group).
    """
    n = L.shape[0]
    if n == 0:
        raise ValueError("The Laplacian matrix is empty.")
    if n == 1:
        return np.array([0]), np.array([])
    if n == 2:
        return np.array([0]), np.array([1])

    # 1) Fiedler vector and sorting
    fiedler = _fiedler_vector(L, solver=solver)
    order = np.argsort(fiedler)

    # 2) Build adjacency in the ordered space and scan best cut
    # A = -L_off; we just reuse your dense scanning logic
    if sp.issparse(L):
        A_perm = (-L)[order][:, order].toarray()
    else:
        A_perm = (-L)[np.ix_(order, order)]

    ind = np.arange(1, n)
    upper_tri_sums = np.array([A_perm[i:, :i].sum() for i in ind], dtype=float)
    qualities = upper_tri_sums / (ind * (n - ind))
    best_cut = int(np.argmin(qualities) + 1)

    g1 = order[:best_cut]
    g2 = order[best_cut:]
    # Keep vertex 0 in the first group as you did
    return (g1, g2) if 0 in g1 else (g2, g1)


# ---------------------------
# Simple benchmark harness
# ---------------------------
def benchmark_bicut(
    num_graphs: int = 5,
    sup: int = 10, sub: int = 10, node: int = 10,  # 10*10*10 = 1000
    p_intra_subgroup: float = 0.8,
    p_intra_supergroup: float = 0.3,
    p_inter_supergroup: float = 0.05,
    solvers: Tuple[str, ...] = ("dense", "sparse"),
    seed_base: int = 1000,
    warmup: bool = True,
) -> List[Dict]:
    """
    Generate several graphs of size sup*sub*node, and time bicut_group with
    different solvers. Returns a list of result dicts and prints a small table.
    """
    results = []
    print(f"\n=== Benchmark bicut (N = {sup*sub*node}, graphs = {num_graphs}) ===")
    print(f"solvers: {', '.join(solvers)}\n")

    # Pre-generate graphs (same graphs for all solvers)
    graphs = []
    for i in range(num_graphs):
        seed = seed_base + i
        L = generate_layers_groups_graph(
            num_supergroups=sup,
            num_subgroups_per_supergroup=sub,
            nodes_per_subgroup=node,
            p_intra_subgroup=p_intra_subgroup,
            p_intra_supergroup=p_intra_supergroup,
            p_inter_supergroup=p_inter_supergroup,
            seed=seed,
        )
        graphs.append(L)

    # Optional warmup to avoid JIT/caches affecting first timing
    if warmup:
        _ = bicut_group(graphs[0], solver="dense")
        _ = bicut_group(graphs[0], solver="sparse")

    # Time each solver on all graphs
    for solver in solvers:
        times = []
        for i, L in enumerate(graphs):
            t0 = time.perf_counter()
            _ = bicut_group(L, solver=solver)
            dt = time.perf_counter() - t0
            times.append(dt)
        avg = float(np.mean(times))
        std = float(np.std(times))
        results.append({"solver": solver, "avg_s": avg, "std_s": std})

    # Pretty print
    print("solver   | avg time (s) | std (s)")
    print("----------------------------------")
    for r in results:
        print(f"{r['solver']:<8}| {r['avg_s']:.4f}       | {r['std_s']:.4f}")
    print()

    return results


# ---------------------------
# Example usage
# ---------------------------
if __name__ == "__main__":
    # Case 1: ~1000 x 1000 (10*10*10)
    benchmark_bicut(num_graphs=5, sup=1, sub=100, node=20, solvers=("dense", "sparse"))

    # You can scale up easily, e.g. ~ 2744 x 2744 (14*14*14)
    # benchmark_bicut(num_graphs=3, sup=14, sub=14, node=14, solvers=("dense", "sparse"))

    # Or make it bigger and sparser by lowering probabilities:
    # benchmark_bicut(num_graphs=3, sup=20, sub=20, node=10,
    #                 p_intra_subgroup=0.4, p_intra_supergroup=0.05, p_inter_supergroup=0.01,
    #                 solvers=("dense", "sparse"))



=== Benchmark bicut (N = 2000, graphs = 5) ===
solvers: dense, sparse

solver   | avg time (s) | std (s)
----------------------------------
dense   | 0.6246       | 0.0102
sparse  | 0.4539       | 0.0133



In [1]:
from tests import *

test_bicut_accuracy(1000,1000,0.8,0.7,0.2)

1.0

In [1]:
from tests import *

run_one_graph_test(1,10,20,1,False,"dense")

{'duration': 0.6040008068084717,
 'memory': 8.014852523803711,
 'ratio': 1.1407710571733556}